# **Домашнее задание №1 (pro). Часть 1**

В этом домашнем задании (его первой части) вам будет необходимо:
*  обучить модель регрессии для предсказания стоимости автомобилей;


> Оценка за первую часть домашки = $min(\text{ваш балл}, 7)$



**Примечание**

В каждой части оцениваются как код, **так и ответы на вопросы.** Вопросы подсвечены синим цветом.

Если нет одного и/или другого, то часть баллов за соответствующее задание снимается.

In [1]:
# !pip install ydata_profiling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
from ydata_profiling import ProfileReport

import re
from phik import phik_matrix
import pickle
import os

random.seed(42)
np.random.seed(42)

/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **Задание 0**
Давайте зафиксируем важный момент.

**Задание 0 (0 баллов).**
Изучите и ответье на вопрос: для чего фиксируем сиды в домашках?

`Сиды фиксируем для воспроизводимости результатов`

# **Часть 1 | EDA**


Первая часть состоит из классических шагов EDA:

- Базовый EDA и обработка признаков (1.7 балла)
- Визуализации признаков и их анализ (1.6 балл)

Всего можно набрать 3.3 основных балла и 0.25 бонусных.

В следующих частях, вы увидите бонусные задания. Бонусные задания выделены как **Дополнительное задание/Бонус**. Вы можете выполнять их, чтобы в случае ошибок в основных задачах всё равно набрать за работу максимум. Кроме того, дополнительные задания позволяют вам углубить знания.

Призываем активно использовать их!

## **Простейший EDA и обработка признаков (1.7 балла)**

In [3]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

Train data shape: (6999, 13)
Test data shape:  (1000, 13)


### **Задание 1.(0.4 балла)**
Выполните операции, направленные на базовое исследование данных:

- [ ] Посмотрите, есть ли в датасете пропуски и дубликаты. Запишите/выведите названия колонок, для которых есть пропущенные значения (0.1 балла)
- [ ] Посмотрите, есть ли в данных явные дубликаты (0.05 балла)
- [ ] Постройте дашборд в одну строку, используя [ydata-profilling](https://github.com/ydataai/ydata-profiling)(0.15 балла)
- [ ] Опишите базовые выводы (какие — выберите сами), используя дашборд (0.2 балла).

In [4]:
df_train.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0


**Ваш ответ здесь.**

In [5]:
# Проверяем пропуски
missing_train = df_train.isna().sum(axis = 0)
missing_test = df_test.isna().sum(axis = 0)

print("Колонки с пропусками в train:")
print(missing_train[missing_train > 0])

print("\nКолонки с пропусками в test:")
print(missing_test[missing_test > 0])

Колонки с пропусками в train:
mileage      202
engine       202
max_power    196
torque       203
seats        202
dtype: int64

Колонки с пропусками в test:
mileage      19
engine       19
max_power    19
torque       19
seats        19
dtype: int64


In [6]:
duplicate_train = df_train.duplicated().sum()
duplicate_test = df_test.duplicated().sum()

print("Кол-во дубликатов в train:", duplicate_train)
print("Кол-во дубликатов в test:", duplicate_test)

Кол-во дубликатов в train: 985
Кол-во дубликатов в test: 62


Достаточно много пропущенных, количество дубликатов не так критично.

In [7]:
profile = ProfileReport(df_train, title="Cars Train Data Profiling")
profile.to_file("train_profile.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 307.12it/s]


Выводы:

* Топ автопроизводителей : maruti, hundai, tata( motors?), mahindra.
Большинство автопроизводителей из Индии. (Выводы из облака слов name)
* 75 % машин моложе 2011 года и 25% моложе 2017 года
* Средняя стоимость машин 639к. 75% машин дешевле 675к. Стоимость топ 5% самых дорогих машин начинается с 1925000.
* Средний пробег 69584 км. У 75%  машин пробег больше 35к
* Большинство машин на бензине и дизеле.
* 83% продавцов - частные лица.  14% - дилеры
* 87% машин на механике
* у 65% машин был лишь 1 владелец
* столбец **mileage** показывает сколько авто может проехать км за 1 л. Но это строковый столбец из-за приписки kmpl. Надо исправить
* столбец **engine** показывает общий объём цилиндров. Но это строковый столбец из-за приписки cc. Надо исправить
* столбец **maxpower** показывает кол-во лошадиных сил. Но это строковый столбец из-за приписки bhp. Надо исправить
* столбец **torque** показывает крутящий момент. Но это строковый столбец из-за приписки rpm. Надо исправить
* 77% машин - 5 местные 13% - 7 местные
* Отрицательная корреляция между годом автомобиля и пробегом - что логично
* Положительная корреляция между годом автомобиля и ценой - чем новее автомобиль тем он дороже, тоже логично


### **Задание 2 (0.2 балла)**
Проанализируйте статистики датасета.

**Ваша задача:**
- [ ] Посчитайте основные статистики по числовым столбцам для трейна и теста
- [ ] Посчитайте основные статистики по категориальным столбцам для трейна и теста
- [ ] Сравните среднее и медиану внутри `train`, внутри `test` и между собой. О чём могут говорить результаты?
- [ ] Сделайте выводы по статистикам. Отсутсвие выводов равносильно по баллам невыполнению задания.

**Подсказка:**

Используте ``.describe()`` с нужным(и) аргументом(-ами).

In [8]:
pd.set_option("display.float_format", "{:.1f}".format)

In [9]:
df_train.describe(include = [np.number])

,year,selling_price,km_driven,seats
count,6999.0,6999.0,6999.0,6797.0
mean,2013.8,639515.2,69584.6,5.4
std,4.1,808941.9,57724.0,1.0
min,1983.0,29999.0,1.0,2.0
25%,2011.0,254999.0,35000.0,5.0
50%,2015.0,450000.0,60000.0,5.0
75%,2017.0,675000.0,97000.0,5.0
max,2020.0,10000000.0,2360457.0,14.0


In [10]:
df_test.describe(include = [np.number])

,year,selling_price,km_driven,seats
count,1000.0,1000.0,1000.0,981.0
mean,2013.7,617901.0,71393.3,5.4
std,4.0,758553.9,48486.2,0.9
min,1995.0,31000.0,1303.0,4.0
25%,2011.0,250000.0,37000.0,5.0
50%,2014.0,434999.0,61500.0,5.0
75%,2017.0,670000.0,100000.0,5.0
max,2020.0,6000000.0,375000.0,9.0


Почти во всех числовых признаках mean > median:
* selling_price (639k > 450k)
* km_driven (69k > 60k)
* seats (5.4 > 5)
* year (2013.8 < 2015 — единственный обратный случай)

**Интерпретации:**

* Признаки имеют *правосторонние* распределения.
* Продажи и пробег содержат редкие большие значения (выбросы).
* Seats: наличие 7–14-местных машин создаёт лёгкий хвост.
* Year: медиана выше → присутствуют старые редкие машины (min 1983).

> **Test**

Поведение аналогичное train, но экстремальные значения слабее.

> **Сравнение train vs test**

* Средние и медианы везде очень близки → **стратификация хорошая**, распределения совпадают.
* Отличие только в:

  * test — немного более старые машины (медиана 2014 vs 2015)
  * test — немного меньшая дисперсия цен и пробегов


In [11]:
df_train.describe(include = [object])

,name,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque
count,6999,6999,6999,6999,6999,6797,6797,6803,6796
unique,1924,4,3,2,5,386,120,316,419
top,Maruti Swift Dzire VDI,Diesel,Individual,Manual,First Owner,18.9 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm
freq,116,3793,5826,6095,4587,197,885,330,468


In [12]:
df_test.describe(include = [object])

,name,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque
count,1000,1000,1000,1000,1000,981,981,981,981
unique,621,4,3,2,5,237,88,182,226
top,Maruti Alto 800 LXI,Diesel,Individual,Manual,First Owner,18.6 kmpl,1248 CC,74 bhp,200Nm@ 1750rpm
freq,15,534,837,877,623,23,116,43,57



Выводы

>**1. Train и test распределены похоже**


> **2. В данных есть  выбросы**

* Цена:  10 млн в train, 6 млн в test
* Пробег: до 2.36 млн км (!)


> **3. Признаки  скошены вправо (кроме year)**



> **4. Категориальные признаки mileage	engine	max_power	torque на самом деле числовые!**




### **Задание 3 (0.2 балла)**

- [ ] Посмотрите, есть ли в трейне объекты с одинаковым признаковым описанием (целевую переменную следует исключить). Если есть, то сколько? (0.01 балла)
- [ ] Отобразите такие объекты (0.01 балла)
- [ ] Удалите повторяющиеся строки. Если при одинаковом признаковом описании цены на автомобили отличаются, то оставьте первую строку по этому автомобилю (0.01 балла)
- [ ]  Обновите индексы строк таким образом, чтобы они шли от 0 без пропусков (0.01 балла)
- [ ] Подумайте, могут ли в данных быть другие скрытые дубли? Предложите, как их можно отлавливать. (0.16 балла)

P.S тут данные без подвоха, но выводы нам нужны.

In [13]:
X_columns = df_train.drop(columns = ['selling_price']).columns

dups = df_train[df_train.duplicated(subset=X_columns, keep='first')]
print('Дублей',dups.shape[0])
dups

Дублей 1159


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
254,Hyundai Grand i10 Sportz,2017,450000,35000,Petrol,Individual,Manual,First Owner,18.9 kmpl,1197 CC,82 bhp,114Nm@ 4000rpm,5.0
258,Maruti Swift VXI,2012,330000,50000,Petrol,Individual,Manual,Second Owner,18.6 kmpl,1197 CC,85.8 bhp,114Nm@ 4000rpm,5.0
268,Maruti Swift Dzire VXI,2014,400000,70000,Petrol,Individual,Manual,Second Owner,19.1 kmpl,1197 CC,85.8 bhp,114Nm@ 4000rpm,5.0
324,Jaguar XE 2016-2019 2.0L Diesel Prestige,2017,2625000,9000,Diesel,Dealer,Automatic,First Owner,13.6 kmpl,1999 CC,177 bhp,430Nm@ 1750-2500rpm,5.0
325,Lexus ES 300h,2019,5150000,20000,Petrol,Dealer,Automatic,First Owner,22.37 kmpl,2487 CC,214.56 bhp,202Nm@ 3600-5200rpm,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6947,Chevrolet Spark 1.0 LS,2010,140000,60000,Petrol,Individual,Manual,Second Owner,18.0 kmpl,995 CC,62 bhp,90.3Nm@ 4200rpm,5.0
6984,Ford Freestyle Titanium Plus Diesel BSIV,2018,746000,24000,Diesel,Individual,Manual,First Owner,24.4 kmpl,1498 CC,98.63 bhp,215Nm@ 1750-3000rpm,5.0
6989,Maruti Swift Dzire VDI,2015,625000,50000,Diesel,Individual,Manual,First Owner,26.59 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
6997,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396 CC,70 bhp,140Nm@ 1800-3000rpm,5.0


In [14]:
df_train = df_train.drop_duplicates(subset=X_columns, keep='first', ignore_index = True)
df_train

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
3,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0
4,Hyundai Xcent 1.2 VTVT E Plus,2017,440000,45000,Petrol,Individual,Manual,First Owner,20.14 kmpl,1197 CC,81.86 bhp,113.75nm@ 4000rpm,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5835,Hyundai Santro Xing GLS,2008,120000,191000,Petrol,Individual,Manual,First Owner,17.92 kmpl,1086 CC,62.1 bhp,96.1Nm@ 3000rpm,5.0
5836,Maruti Wagon R VXI BS IV with ABS,2013,260000,50000,Petrol,Individual,Manual,Second Owner,18.9 kmpl,998 CC,67.1 bhp,90Nm@ 3500rpm,5.0
5837,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197 CC,82.85 bhp,113.7Nm@ 4000rpm,5.0
5838,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493 CC,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0


In [15]:
df_train.head(10)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
3,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0
4,Hyundai Xcent 1.2 VTVT E Plus,2017,440000,45000,Petrol,Individual,Manual,First Owner,20.14 kmpl,1197 CC,81.86 bhp,113.75nm@ 4000rpm,5.0
5,Maruti Wagon R LXI DUO BSIII,2007,96000,175000,LPG,Individual,Manual,First Owner,17.3 km/kg,1061 CC,57.5 bhp,"7.8@ 4,500(kgm@ rpm)",5.0
6,Maruti 800 DX BSII,2001,45000,5000,Petrol,Individual,Manual,Second Owner,16.1 kmpl,796 CC,37 bhp,59Nm@ 2500rpm,4.0
7,Toyota Etios VXD,2011,350000,90000,Diesel,Individual,Manual,First Owner,23.59 kmpl,1364 CC,67.1 bhp,170Nm@ 1800-2400rpm,5.0
8,Ford Figo Diesel Celebration Edition,2013,200000,169000,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,68.1 bhp,160Nm@ 2000rpm,5.0
9,Renault Duster 110PS Diesel RxL,2014,500000,68000,Diesel,Individual,Manual,Second Owner,19.01 kmpl,1461 CC,108.45 bhp,248Nm@ 2250rpm,5.0


Высока вероятность, что скрытые дубли могут возникнуть из-за названия. Например, не полностью указал модель. Указал с маленькой буквы и тд. Также одна машина  могла продаваться несколько раз в разные периоды времени - поменялся пробег и число владельцев, но машина та же. Но нас интересут цена, поэтому это не так важно.
Разная запись расхода kmpl или km/kg
Человек мог сначала сам продавать машину, потом продать её диллеру и уже диллер будет её продавать.

In [16]:
assert df_train.shape == (5840, 13)

### **Задание 4 (0.3 балла)**

Вы могли заметить, что с признаками ``mileage, engine, max_power и torque`` всё не очень хорошо. Они распознаются как строки (можно убедиться в этом, вызвав `data.dtypes`). Однако эти переменные не являются категориальными — они — числа. Соответственно, нужно привести их к числовому виду.

**Задача :**
* [ ] Уберите единицы измерения для признаков ``mileage, engine, max_power``.
* [ ] Приведите тип данных к ``float``.
* [ ] Предобработайте признак `torque` — разделите его на два: собственно `torque` и `max_torque_rpm`. Учтите единицы измерения


**Важно**
- Все действия нужно производить над обоими датасетами — `train` и `test`.

In [17]:
def clean_word(x):
    if pd.isna(x):
        return np.nan
    x = str(x).lower()
    x = x.replace("kmpl", "").replace("km/kg", "").replace("cc", "").replace("bhp", "").strip()
    return float(x) if x else np.nan


def parse_torque(t):
    """
    Возвращает (torque_Nm, rpm) как два числа.
    Обрабатываем варианты:
    - "190Nm@ 2000rpm"
    - "22.4 kgm at 1750-2750rpm"
    - "11.5@ 4,500(kgm@ rpm)"
    - "250Nm@ 1500-2500rpm"
    - "114@ 3,000rpm"
    """

    if pd.isna(t):
        return pd.Series([np.nan, np.nan])

    t = str(t).lower()

    t = t.replace(",", "").replace(" ", "").replace("(kgm@rpm)", "").replace("kgm@", "kgm").replace("nm@", "nm")

    # Достаём крутящий момент
    # варианты: "190nm", "22.4kgm", "11.5"
    torque_match = re.findall(r"([\d\.]+)(nm|kgm)?", t)
    if torque_match:
        value, unit = torque_match[0]
        value = float(value)

        # конвертация kgm → Nm
        if unit == "kgm":
            value = value * 9.80665  # физическая константа

    else:
        value = np.nan

    # Достаём rpm (берём минимальное значение диапазона)
    rpm_match = re.findall(r"(\d+)-?(\d+)?rpm", t)
    if rpm_match:
        rpm1, rpm2 = rpm_match[0]
        rpm = int(rpm1)
    else:
        # формат типа "@4500"
        rpm_match = re.findall(r"@(\d+)", t)
        rpm = int(rpm_match[0]) if rpm_match else np.nan

    return pd.Series([value, rpm])


def preprocess(df):
    df = df.copy()
    df['mileage'] = df['mileage'].apply(clean_word)
    df['engine'] = df['engine'].apply(clean_word)
    df['max_power'] = df['max_power'].apply(clean_word)

    df[['torque', 'max_torque_rpm']] = df['torque'].apply(parse_torque)

    df['mileage'] = df['mileage'].astype(float)
    df['engine'] = df['engine'].astype(float)
    df['max_power'] = df['max_power'].astype(float)
    df['torque'] = df['torque'].astype(float)
    df['max_torque_rpm'] = df['max_torque_rpm'].astype(float)

    return df

In [18]:
df_train = preprocess(df_train)
df_test = preprocess(df_test)

### **Задание 5 (0.3 балла)**

На первом шаге мы обнаружили пропуски. Давайте избавимся от них.

**Задание:**
- [ ] Заполните пропуски в столбцах медианами. Убедитесь, что после заполнения пропусков не осталось.
- [ ] Почему стоит применять именно медиану. Могли ли мы применить среднее? Обоснуйте свое рассуждение.
- [ ] Как правильно считать медиану для заполнения? Выберите верное утверждение:
 - По тестовым свою, по тренировочным — свою
 - По тренировочным данным для `train` и `test`






Медиана не реагирует на выбросы, поэтому она:

* даёт более стабильную оценку центра

* не искажает данные

* меньше портит распределения признаков

Правильно заполнять по тренировочным данным для train и test, чтобы не было утечки данных. Если  брать медиану из теста, мы «сообщаем» модели информацию о совокупности данных, которую она не должна знать.
Модель начинает работать с данными, которые были статистически «подправлены» под текущий тестовый набор, а не под то, чему она училась.



In [19]:
for col in ["mileage", "engine", "max_power", "torque", "max_torque_rpm", "seats"]:
    med = df_train[col].median()
    df_train[col] = df_train[col].fillna(med)
    df_test[col]  = df_test[col].fillna(med)

In [20]:
print('Кол-во пропусков в train')
df_train.isna().sum().sum().item()

Кол-во пропусков в train


0

In [21]:
print('Кол-во пропусков в test')
df_test.isna().sum().sum().item()

Кол-во пропусков в test


0

### **Задание 6 (0.2 балла)**

Теперь, когда не осталось пропусков, давайте преобразуем столбцы к более подходящим типам. А именно столбцы ``engnine`` и ``seats`` к приведем к `int`.

- [ ] Осуществите приведение столбцов к необходимому типу.
- [ ] Ответье на вопрос — почему (хоть мы этого и не делаем) ``seats``, возможно рассмотреть как категориальную переменную?

In [22]:
df_train["engine"] = df_train["engine"].astype(int)
df_test["engine"]  = df_test["engine"].astype(int)

df_train["seats"] = df_train["seats"].astype(int)
df_test["seats"]  = df_test["seats"].astype(int)

In [23]:
df_train["seats"].unique()

array([ 5,  4,  7,  8,  6,  9, 10, 14,  2])


seats принимает ограниченный набор значений:
2, 4, 5, 6, 7, 8, 9, 10, 14

Эти значения — не непрерывный числовой ряд, а категории по вместимости автомобиля:

2 — спорткары, купе

4 — миникомпактные

5 — стандартные седаны / хэтчбеки

6–7 — кроссоверы, минивэны

8–14 — микроавтобусы


### **Задание 7 (0.1 балла)**

Снова вызовите метод describe и проанализируйте статистики.

**Ответье на вопрос:**
- [ ] Есть ли основания предполагать, что заполнение пропусков свдинуло наши распределения? Могло ли это вообще возникнуть?

In [24]:
df_train.describe()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,max_torque_rpm
count,5840.0,5840.0,5840.0,5840.0,5840.0,5840.0,5840.0,5840.0,5840.0
mean,2013.4,522960.1,73952.2,19.4,1429.4,87.9,163.6,5.4,2654.8
std,4.1,535432.0,60071.1,4.0,485.7,31.6,89.7,1.0,1118.6
min,1983.0,29999.0,1.0,0.0,624.0,0.0,5.7,2.0,175.0
25%,2011.0,250000.0,39000.0,16.9,1197.0,68.0,102.0,5.0,1750.0
50%,2014.0,405000.0,70000.0,19.4,1248.0,81.9,153.0,5.0,2000.0
75%,2017.0,640000.0,100000.0,22.3,1498.0,99.0,200.0,5.0,3750.0
max,2020.0,10000000.0,2360457.0,42.0,3604.0,400.0,789.0,14.0,21800.0


Во первых наблюдений в трейне стало меньше из=за удаления дубликатов. Мы видим, что средняя цена продажи и её медиана упали, поэтому вероятно, что было больше дублей машин которые стоили больше среднего. Пробег в целом тоже увеличился. Кол-во мест осталос прежним

In [25]:
df_test.describe()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,max_torque_rpm
count,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mean,2013.7,617901.0,71393.3,19.3,1454.9,90.8,168.4,5.4,2690.8
std,4.0,758553.9,48486.2,4.0,522.0,34.9,93.7,0.9,1115.4
min,1995.0,31000.0,1303.0,0.0,624.0,34.2,7.8,4.0,500.0
25%,2011.0,250000.0,37000.0,16.6,1197.0,69.0,104.0,5.0,1750.0
50%,2014.0,434999.0,61500.0,19.4,1248.0,82.4,153.0,5.0,2000.0
75%,2017.0,670000.0,100000.0,22.3,1582.0,102.0,205.0,5.0,4000.0
max,2020.0,6000000.0,375000.0,32.3,3604.0,280.0,620.0,9.0,5000.0


test мы не трогали - появились лишь новые числовые столбцы которые мы преобразовали из object

`Кол-во пропусков было маленьким, примерно 200/5800 = 3%, поэтому на распределение это повлияло незначительно. Возможно, немного увеличило пик около медианы`

## **Визуализации и корреляция (1.6 балла + 0.25)**

Визуализация данных — важный шаг в работе. Визуализировать данные необходимо, например, чтобы:

- Оценить распределения признаков самих по себе (это может натоклнуть вас на мысли о модели, которую можно использовать)
- Сравнить распределения на `train` и `test` — чтобы проверить, насколько информация, на которой вы будете обучаться согласуется с той, на которой модель должна работать
- Оценить есть ли явная связь признаков с целевой переменной

**Важно:**

Если распределения на `train` и `test` не совпадают, это не значит, что нужно перемешивать данные! Более корректно актуализировать задачу и уточнить, а не устарели ли данные `train`. Также полезным может быть собрать новую тестовую выборку, смешав те, что имеются сейчас.

**Если вы будете подгонять распределения, то можете встретиться с переобучением!**

### **Задание 8 (0.5 балла)**

Шаг 1.
- [ ] Воспользуйтесь `pairplot` из библиотеки `seabron`, чтобы визуализировать попарные распределения числовых признаков для `train`
- [ ] По полученному графику ответьте на вопросы:
 - Можно ли предположить на основе распределений связь признаков с целевой переменной?
 - Можно ли предположить на основе распределений выдвинуть гипотезу о корреляциях признаков?

Шаг 2.

- [ ] Постройте pairplot по тестовым данным
- [ ] Ответьте на вопрос "Похожими ли оказались совокупности при разделении на трейн и тест?"

In [26]:
num_cols = df_train.select_dtypes(include=['int64', 'float64']).columns

sns.pairplot(df_train[num_cols]);

Можно заметить следующее:
цена увеличивается при увеличении: year, engine, max_power

цена уменьшается при увеличении: km_driven



Корреляции между признаками — тоже есть:

сильнее всего связь: engine и max_power, torque и max_power

заметна связь : engine и torque



In [27]:
sns.pairplot(df_test[num_cols]);

Бросается в глаза то, что шкала km_driven перестала быть 1e6. В трейне она была таковой  из-за выбросов. Там была машина с пробегом 2 млн км :)   По этой причине графиг более растянутый и понятный - но само распределение это не меняет.

Также шкала поменялась у selling price с 1e7 on train на 1e6 на test

В целом графики могли немного сместиться из-за отличий в шкалах. На трейне больше вероятность встретить выброс/крупные или очень маленькие значения - и шкала меняется соответствующе. Попробуем не учитывать первый и последний перцентиль и построить графики:

In [28]:
def clip_by_quantiles(df, cols, q_low=0.01, q_high=0.99):
    df = df.copy()
    for col in cols:
        low = df[col].quantile(q_low)
        high = df[col].quantile(q_high)
        df[col] = df[col].clip(lower=low, upper=high)
    return df


df_clipped = clip_by_quantiles(df_train[num_cols], num_cols)

sns.pairplot(df_clipped);

/var/folders/qh/bc52z7wx0mzgnf9wlwvfw9sh0000gn/T/ipykernel_10395/320896347.py:6: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].clip(lower=low, upper=high)


In [29]:
df_clipped_test = clip_by_quantiles(df_test[num_cols], num_cols)
sns.pairplot(df_clipped_test)

/var/folders/qh/bc52z7wx0mzgnf9wlwvfw9sh0000gn/T/ipykernel_10395/320896347.py:6: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].clip(lower=low, upper=high)


Можно заметить следующее: цена увеличивается при увеличении: year, engine, max_power и **rougue**

цена уменьшается при увеличении: km_driven

Корреляции между признаками — тоже есть:

сильнее всего связь: engine и max_power, torque и max_power

заметна связь : engine и torque

Заметно, что чем мощнее двигатель, тем меньше mileage, то есть тем меньше проедешь на одном литре

>Распределение train и test похоже

### **Задание 9 (0.5 балла)**

И так, вы выдвинули гипотезы о наличии связи. Теперь давайте оценим эту связь в числах.

**Задание:**
- [ ] Получите значения коэффициента корреляции Пирсона для тренировочного набора данных при помощи `pd.corr()`
- [ ] По полученным корреляциям постройте тепловую карту (`heatmap` из бибилотеки seaborn)

In [30]:
corr = df_train.corr(numeric_only=True)
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm");

- [ ] Ответьте на вопросы:
 - Какие 2 признака наименее скоррелированы между собой?
 - Между какими наблюдается довольно сильная положительная линейная зависимость?
 - Правильно ли, опираясь на данные, утверждать, что чем меньше год, тем, скорее всего, больше километров проехала машина к дате продажи?
 - Изучите типы корреляций в `pd.corr()`. Какую вы использовали по умолчанию?


Минимальный по модулю коэффициент  между признаками **engine** и **year** (приблизительно 0.00). Это означает, что между ними практически отсутствует линейная зависимость.

---

Наиболее высокая положительная корреляция среди различных пар признаков наблюдается между **max_power** и **torque** (порядка 0.76). Это указывает на выраженную линейную зависимость: автомобили с более высокой мощностью, как правило, обладают и большим крутящим моментом.

---
Наиболее высокая отрицательная корреляция среди различных пар признаков наблюдается между **engine** и **mileage** (-0.57). Это говорит о том, что чем мощнее двигатель, тем меньше можно проехать на 1 литр топлива
---
Корреляция между признаками **year** и **km_driven** составляет примерно –0.37, что соответствует умеренной отрицательной зависимости. Это означает, что более старые автомобили действительно чаще имеют больший пробег. На основании этих данных можно сделать вывод о наличии такой тенденции. Однако стоит учитывать, что это статистическая зависимость и возможны отдельные исключения.

---
Метод `pandas.DataFrame.corr()` по умолчанию использует **корреляцию Пирсона**. Данный тип корреляции измеряет линейную зависимость между числовыми переменными. Также можно считать корреляцию Кендела и Спирмена. Они помогают уловить в том числе нелинейные зависимости



### **Задание 10 (0.6 балла)**

По умолчанию `pd.corr` возвращает корреляцию Пирсона, говорящую о линейной взаимосвязи. Но зависимости существуют не только линейные! В этой задаче, попробуйте измерить другие способы вычисления корреляций.

- [ ] Реализуйте корреляцию Спирмена/Кендала (на выбор) без использования библиотек (можно пользоваться только `numpy`). Сравните результаты вычисления с библиотечной реализацией
- [ ] Сделайте выводы


Постройте матрицу корреляции [phik](https://pypi.org/project/phik/)
- [ ] Проинтерпретируйте результаты


In [31]:
def rank_average(x):
    x = np.asarray(x)
    order = np.argsort(x)
    ranks = np.empty(len(x), dtype=float)

    i = 0
    while i < len(x):
        j = i
        while j + 1 < len(x) and x[order[j]] == x[order[j+1]]:
            j += 1

        avg_rank = (i + j) / 2
        ranks[order[i:j+1]] = avg_rank
        i = j + 1

    return ranks


def spearman(df):
    n, m = df.shape
    corr = np.zeros((m, m))

    ranks = np.zeros((n, m))
    for idx, col in enumerate(df.columns):
        ranks[:, idx] = rank_average(df[col].values)

    # Спирмен = Пирсон по рангам
    for i in range(m):
        for j in range(m):
            corr[i, j] = np.corrcoef(ranks[:, i], ranks[:, j])[0, 1]

    return corr

In [32]:
corr_pandas = df_train.corr(numeric_only=True, method = 'spearman')
plt.figure(figsize=(12, 8))
sns.heatmap(corr_pandas, annot=True, fmt=".2f", cmap="coolwarm");

In [33]:
np.allclose(corr_pandas, spearman(df_train.loc[:, num_cols]))

True

`Моя реализация совпадает с библиотечной`

Главные отличия Пирсон vs Спирмен :

* year и selling_price — Пирсон даёт всего 0.43, а Спирмен 0.71. Год выпуска на самом деле ОЧЕНЬ сильно (и нелинейно) влияет на цену.
* year и km_driven — с 0.37 до 0.57. Пробег тоже намного сильнее (и нелинейно) зависит от года.
* Обороты максимального крутящего момента (max_torque_rpm) с объёмом/мощностью/крутящим моментом — Спирмен видит заметно более сильную отрицательную связь
* Количество мест (seats) с ценой — Пирсон почти не видит (0.15), Спирмен уже умеренная (0.32).


In [34]:
phik_corr = df_train.phik_matrix(interval_cols=num_cols)
plt.figure(figsize=(12, 8))
sns.heatmap(phik_corr, annot=True, fmt=".2f", cmap="coolwarm");

/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/phik/data_quality.py:59: UserWarning: The number of unique values of variable name is large: 1924. Are you sure this is not an interval variable? Analysis for pairs of variables including name can be slow.
  warnings.warn(



- **name** — это модель машины (у неё тысячи уникальных значений).  
- Почти у каждой модели уникальное название и название модели и её характеристики, включая стоимость неразрывно связаны, потому у данной переменной такая высокая корреляция со всеми.

Топ по влиянию на **selling_price** :

1. **year** — 0.92
2. **max_power** — 0.87  
3. **torque** — 0.75  
4. **engine** — 0.72  
5. **mileage** — 0.65  
6. **owner** — 0.56  
7. **transmission** — 0.46 (автомат сильно дороже)  
8. **fuel** — 0.33 (дизель чуть дороже)  
9. **seats** — 0.20  
10. **km_driven** —  0.00 (Неожиданно, так как по Пирсону и Спирману там была корреляцию, но отрицательная)  


---
самые сильные взаимосвязи между признаками :

1. **torque и engine** — 0.88 — тоже почти дублируют друг друга
2. **max_power и torque** — 0.86 — почти одно и то же  
3. **fuel и max_torque_rpm** — 0.85
  
4. **mileage и engine** — 0.69 — чем больше двигатель, тем выше расход  
5. **max_power и mileage** — 0.69 — мощные машины жрут больше  
6. **torque и mileage** — 0.72 — аналогично  
7. **max_power и engine** — 0.72
8. **torque и max_torque_rpm** — 0.65

группа **engine, max_power, torque, mileage и torque_rpm** — это похожие по сути признаки. Может быть лучше будет удалить несколько


### **Дополнительные визуализации (бонус 0.25 балла)**

Если вам кажется, что мы не попросили вас нарисовать какие-то очень важные зависимости, нарисуйте их и поясните.
Один график: 0.125 балла, при условии, что он обоснован.

In [35]:
num_cols
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    sns.boxplot(y=df_train[col], ax=axes[i], color='lightblue', boxprops=dict(alpha=0.8))
    axes[i].set_title(f'Boxplot: {col}', fontsize=14, fontweight='bold')
    axes[i].set_ylabel(col, fontsize=12)
    axes[i].grid(True, alpha=0.3)

`Как видно из графиков. У нас в данных есть достаточное кол-во выбросов, на что также стоит обратить внимание.`

In [36]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(20, 15))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    sns.histplot(data=df_train, x=col, kde=True, ax=axes[i], color='skyblue', alpha=0.7, bins=50)
    axes[i].set_title(f'Распределение {col}', fontsize=14, fontweight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Частота')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Распределения числовых признаков (с KDE)', fontsize=16, fontweight='bold', y=1.02)
plt.show()

/var/folders/qh/bc52z7wx0mzgnf9wlwvfw9sh0000gn/T/ipykernel_10395/1663132956.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


`Видно что распределения цены машины и пробега сильно скошены и чтобы наша модель лучше обучалась (их распределение было больше похоже на нормальное) возможно стоит логарифмировать их.`

# **Часть 2 (1.7 балла) | Модель только на вещественных признаках**

В этой части вам предстоит обучить модель только на вещественных признаках. Почему только на них?

Чем больше признаковое пространство — чем сложнее модель. А чем модель проще — тем лучше для скорости работы и интерпретации признаков.

За задания этой части вы можете набрать 1.7 основных и 0.15 бонусных балла;

### **Задание 11 (0.05 балла)**

Сделайте на тренировочный и тестовый наборы. Она уже даны, достаточно просто отделить целевой признак. Перед разбиением создайте копию датафрейма, который будет хранить только вещественные признаки и используйте его (то есть категориальные столбцы (все, кроме seats) необходимо удалить).

В переменные y_train и y_test запишите значения целевых переменных. Используйте уже имеющиеся данные train и test. Если считаете уместным применить какую-то более хитрую технику разбиения — примените, но опишите, зачем.

In [37]:
df_train_num = df_train.loc[:, num_cols]
df_test_num = df_test.loc[:, num_cols]

X_train_num = df_train_num.drop(columns = ['selling_price'])
X_test_num = df_test_num.drop(columns = ['selling_price'])

y_train = df_train[['selling_price']]
y_test= df_test[['selling_price']]

### **Задание 12. (0.2 балла)**

Построим нашу первую модель!
- [ ] Обучите классическую линейную регрессию с дефолтными параметрами. Посчтитайте $R^2$ и $MSE$ для трейна и для теста.
- [ ] Сделайте выводы по значениям метрик качества.

**Примечание:**

Здесь и далее $R^2$ и $MSE$ для трейна и для теста выводите везде, где требуется обучать модели, даже если в явном виде этого не просят. Иначе непонятно, как понять, насколько успешны наши эксперименты.

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error as MSE

model1 = LinearRegression()
model1.fit(X_train_num, y_train);

In [39]:
def evaluate(model, X_train, X_test, y_train, y_test, model_name="Model1"):
    y_train_pred = model.predict(X_train)
    y_test_pred  = model.predict(X_test)

    train_r2   = r2_score(y_train, y_train_pred)
    train_mse  = MSE(y_train, y_train_pred)

    test_r2   = r2_score(y_test, y_test_pred)
    test_mse  = MSE(y_test, y_test_pred)


    # Простой и чистый вывод
    print(f"Метрики {model_name}:\n")
    print('Train')
    print(f"  R2   = {train_r2:.4f}")
    print(f"  MSE  = {train_mse:.3f}")
    print(f"  RMSE  = {train_mse**0.5:.3f}")


    print("-" * 25)
    print('Test')
    print(f"  R2   = {test_r2:.4f}")
    print(f"  MSE  = {test_mse:.3f}")
    print(f"  RMSE  = {test_mse**0.5:.3f}")

In [40]:
evaluate(model1, X_train_num, X_test_num, y_train, y_test)

Метрики Model1:

Train
  R2   = 0.6016
  MSE  = 114201344891.402
  RMSE  = 337936.895
-------------------------
Test
  R2   = 0.5986
  MSE  = 230742570359.133
  RMSE  = 480356.712


480356 рупий - это примерно 420 к рублей. Такая ошибка - слишком большая для задачи предсказания цены. Особенно учитывая, что половина машин в тесте стоит меньше 434 к рупий

Но так как r2 выше 0 - это значит, что мы определённо лучше предсказания константой(средним по таргету). Мы объяснили 60% дисперсии цены - это достаточно слабый результат

### **Задание 13 (0.15 балла)**

- [ ] Реализуйте $R^2$ руками. Приведите формулу $R^2$ и объясните каждую компоненту метрики

In [41]:
def r2_my(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Среднее значение реальных значений (y_mean)
    y_mean = np.mean(y_true)

    # Total Sum of Squares — общая дисперсия (как сильно y отклоняется от среднего)
    SS_tot = np.sum((y_true - y_mean) ** 2)

    # Residual Sum of Squares — необъяснённая дисперсия (ошибки модели)
    SS_res = np.sum((y_true - y_pred) ** 2)

    # R2 = 1 - (необъяснённая дисперсия / общая дисперсия)
    r2 = 1 - (SS_res / SS_tot)

    return r2

In [42]:
y_train_pred = model1.predict(X_train_num)
y_test_pred  = model1.predict(X_test_num)

train_r2   = r2_my(y_train, y_train_pred)
test_r2   = r2_my(y_test, y_test_pred)
print(f"  R2 train   = {train_r2:.4f}")
print(f"  R2 test  = {test_r2:.4f}")

  R2 train   = 0.6016
  R2 test  = 0.5986


`Результаты совпадают с метрикой sklearn`

### **Бонус (0.15 балла)**

- [ ] Реализуйте [$\text{adjusted}-R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination).
- [ ] Объясните, когда применяется $\text{adjusted}-R^2$?

In [43]:
def adjusted_r2(y_true, y_pred, n_features = X_train_num.shape[1]):
    n = len(y_true)                    # количество объектов
    p = n_features                     # количество признаков (переменных)

    r2 = r2_my(y_true, y_pred)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    return adjusted_r2

In [44]:
train_r2_adj   = adjusted_r2(y_train, y_train_pred)
test_r2_adj   = adjusted_r2(y_test, y_test_pred)
print(f"  R2 adj train   = {train_r2_adj:.4f}")
print(f"  R2 adj test  = {test_r2_adj:.4f}")

  R2 adj train   = 0.6010
  R2 adj test  = 0.5953


Обычный R2 имеет критический недостаток: он **монотонно возрастает** при добавлении любого нового признака, даже бесполезного. Это создаёт иллюзию улучшения модели.

Adjusted R² решает эту проблему, вводя **штраф за количество признаков**:

$$
\text{Adjusted } R^2 = 1 - \frac{(1 - R^2)(n-1)}{n-p-1}
$$

- Если новый признак даёт прирост R² меньше, чем величина штрафа — Adjusted R² уменьшается
- Это позволяет объективно сравнивать модели с разным числом признаков


### **Задание 14 (0.05 балла)**

Всегда есть место совершенству. Поэтому давайте попробуем улучшить модель. При помощи стандартизации признаков.

- [ ] Стандартизируйте значения в тренировочных и тестовых данных. Стандартизатор **обучайте только на `train`**.

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)
X_test_scaled  = scaler.transform(X_test_num)

In [46]:
model2 = LinearRegression()
model2.fit(X_train_scaled, y_train)
evaluate(model2, X_train_scaled, X_test_scaled, y_train, y_test, "Model2")

Метрики Model2:

Train
  R2   = 0.6016
  MSE  = 114201344891.403
  RMSE  = 337936.895
-------------------------
Test
  R2   = 0.5986
  MSE  = 230742570359.134
  RMSE  = 480356.712


### **Задание 15 (0.1 балла)**

Хотя стандартизация не помогла сильно прибавить в качестве она открыла возможность интерпретировать важность признаков в модели. Правило интерпретации такое:

Чем больше коэффициент $\beta_i$ по модулю, тем важнее признак.

**Ответьте на вопрос:**

- [ ] Какой признак оказался наиболее информативным в предсказании цены?

In [47]:
def best_coefs(model, features, n = 10):
    coefs = pd.DataFrame({
        'Признак': features,
        'Коэффициент': model.coef_.flatten()
    })

    coefs = coefs.sort_values('Коэффициент', ascending=False, key = lambda x : abs(x)).head(n)
    print("\n------Коэффициенты------")
    print(coefs.round(3))

In [48]:
best_coefs(model2, X_train_num.columns)


------Коэффициенты------
          Признак  Коэффициент
4       max_power     300158.5
0            year     159616.9
1       km_driven     -49040.9
7  max_torque_rpm     -48733.1
5          torque      36847.7
6           seats     -33559.5
3          engine      17658.6
2         mileage      -8409.1


Наиболее важный признак - max_power

### **Задание 16 (0.25 балла)**

Попробуем улучшить нашу модель с помощью применения регуляризации. Для этого воспльзуемся `Lasso` регрессией.  Кроме того, попробуйте использовать её теоретическое свойство отбора признаков, за счет зануления незначимых коэффициентов.

**Задание:**

- [ ] Обучите Lasso регрессию на тренировочном наборе данных с нормализованными признаками. Оцените её качество
- [ ] Проверьте, занулила ли L1-регуляризация с параметрами по умолчанию какие-нибудь веса? Предположите почему.

In [49]:
from sklearn.linear_model import Lasso

model3 = Lasso(alpha=1.0, random_state=42)
model3.fit(X_train_scaled, y_train)


evaluate(model3, X_train_scaled, X_test_scaled, y_train, y_test, "Lasso default")

best_coefs(model3, X_train_num.columns)

Метрики Lasso default:

Train
  R2   = 0.6016
  MSE  = 114201344904.384
  RMSE  = 337936.895
-------------------------
Test
  R2   = 0.5986
  MSE  = 230743276541.301
  RMSE  = 480357.447

------Коэффициенты------
          Признак  Коэффициент
4       max_power     300159.8
0            year     159614.8
1       km_driven     -49040.0
7  max_torque_rpm     -48730.9
5          torque      36847.8
6           seats     -33555.5
3          engine      17656.7
2         mileage      -8405.9


In [50]:
model4 = Lasso(alpha=5000.0, random_state=42)
model4.fit(X_train_scaled, y_train)


evaluate(model4, X_train_scaled, X_test_scaled, y_train, y_test, "Lasso default")
best_coefs(model4, X_train_num.columns)

Метрики Lasso default:

Train
  R2   = 0.6007
  MSE  = 114452485329.531
  RMSE  = 338308.270
-------------------------
Test
  R2   = 0.5921
  MSE  = 234469873104.504
  RMSE  = 484220.893

------Коэффициенты------
          Признак  Коэффициент
4       max_power     305416.4
0            year     151698.2
1       km_driven     -44062.3
7  max_torque_rpm     -40434.5
5          torque      38020.2
6           seats     -15253.3
3          engine       3985.2
2         mileage         -0.0


Ничего не занулилось с a = 1, так как это слишком маленький коефицент, при а = 5000 занулился mileage

### **Задание 17. Финальный рывок (0.4 балла)**

До этого мы с вами использовали `train` для обучения и `test` для прогнозирования. Но у нас есть ещё одна задача — подобрать оптимальные параметры модели. Для этого используем кросс-валидацию, описанную на семинарах.

Кроме того, выжмем максимум из модификаций регрессии. Построим `ElasticNet`. И сделаем всё по порядку.

**Ваша задача 1:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для Lasso-регрессии. Вам пригодится класс [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
- [ ] Ответьте на вопросы:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Что значит каждый параметр `param_grid`, который вы заполняли для `GridSearch`? Опишите каждый
 - Какой коэффициент регуляризации у лучшей из перебранных моделей? Занулились ли какие-нибудь из весов при такой регуляризации?

In [51]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [1, 5, 10, 100] + list(range(1000, 30000, 500))
}

grid1 = GridSearchCV(
    Lasso(random_state=42, max_iter=10000),
    param_grid,
    cv=10,
    scoring='r2',
    n_jobs=-1
)

grid1.fit(X_train_scaled, y_train);

In [52]:
grid1.best_estimator_

,alpha,22500
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,10000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,42
,selection,'cyclic'


In [53]:
len(grid1.param_grid['alpha'])

62

In [54]:
best_coefs(grid1.best_estimator_, X_train_num.columns)


------Коэффициенты------
          Признак  Коэффициент
4       max_power     286413.6
0            year     140137.7
5          torque      49097.7
1       km_driven     -26743.3
7  max_torque_rpm     -13980.0
2         mileage          0.0
3          engine          0.0
6           seats         -0.0


* грид-сёрчу пришлось обучать 10*62 = 620 моделей
* Я заполнил лишь праметр alpha отвечающий за силу регуляризации
* У лучшей модели alpha = 22500. Она занулила mileage, engine, seats

In [55]:
evaluate(grid1.best_estimator_, X_train_scaled, X_test_scaled, y_train, y_test, "Grid 1")

Метрики Grid 1:

Train
  R2   = 0.5936
  MSE  = 116496920455.930
  RMSE  = 341316.452
-------------------------
Test
  R2   = 0.5720
  MSE  = 246020129510.113
  RMSE  = 496004.163


**Ваша задача 2:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) регрессии.
- [ ] Ответьте на вопрос:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Какие гиперпараметры соответствуют лучшей (по выбранной метрике качества) из перебранных моделей?

In [56]:
from sklearn.linear_model import ElasticNet

param_grid = {
    'alpha': range(15000, 30000, 500),
    'l1_ratio' : np.arange(0, 1.01, 0.1)
}

grid2 = GridSearchCV(
    ElasticNet(random_state=42, max_iter=5000),
    param_grid,
    cv=10,
    scoring='r2',
    n_jobs=-1
)

grid2.fit(X_train_scaled, y_train);

/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.393e+14, tolerance: 1.479e+11
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.602e+14, tolerance: 1.521e+11
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solver

In [57]:
len(np.arange(0, 1.01, 0.1)) * len(range(15000, 30000, 500))

330

In [58]:
grid2.best_estimator_

,alpha,22500
,l1_ratio,1.0
,fit_intercept,True
,precompute,False
,max_iter,5000
,copy_X,True
,tol,0.0001
,warm_start,False
,positive,False
,random_state,42
,selection,'cyclic'


In [59]:
best_coefs(grid2.best_estimator_, X_train_num.columns)


------Коэффициенты------
          Признак  Коэффициент
4       max_power     286413.6
0            year     140137.7
5          torque      49097.7
1       km_driven     -26743.3
7  max_torque_rpm     -13980.0
2         mileage          0.0
3          engine          0.0
6           seats         -0.0


* грид-сёрчу пришлось обучать 10*330 = 3300 моделей
*  alpha - общая сила регуляризации
*  l1 ratio - доля l1 регуляризации в общей

* У лучшей модели alpha = 22500. l1 ratio = 1. По сути это ничем не отличается от предыдущей модели. Она занулила mileage, engine, seats

In [60]:
evaluate(grid2.best_estimator_, X_train_scaled, X_test_scaled, y_train, y_test, "Grid 2")

Метрики Grid 2:

Train
  R2   = 0.5936
  MSE  = 116496920455.930
  RMSE  = 341316.452
-------------------------
Test
  R2   = 0.5720
  MSE  = 246020129510.113
  RMSE  = 496004.163


### **Задание 18 (0.35 балла)**

И так, вы обучили все варианты регуляризаций, доступные в sklearn, но не все, что мы обсудили на занятии! И следующее задание о забытой - $L0$ регуляризации.


**Ваша задача:**>
- [ ] Реализуйте модель с $L0$-регуляризаицей.
- [ ] Обучите модель на данных и проанализируйте результат, экспериментируя с коэффициенитом регуляризации
- [ ] Проанализируйте результаты обучения

**Важно:**

Вопреки тому, что L0 не реализована в стандартных пакетах, концепция данной регуляризации не является мертвой. Она может встречаться в [статьях](https://arxiv.org/abs/1712.01312) и экспериментах.



In [61]:
def calculate_aic_bic_multi(y_true, y_pred, n_features):
    n = len(y_true)
    k = n_features
    if len(y_true.shape) > 1:
        n_targets = y_true.shape[1]
    else:
        n_targets = 1

    mse = MSE(y_true, y_pred, multioutput='uniform_average')

    # Log-likelihood (упрощенная формула для гауссовского распределения)
    # log_likelihood = -n * n_targets / 2 * (np.log(2 * np.pi * mse) + 1)
    # Можно использовать упрощенную формулу для AIC/BIC через MSE:
    # AIC = n * ln(MSE) + 2 * k

    log_likelihood = -n * n_targets / 2 * (np.log(2 * np.pi * mse) + 1)

    total_params = n_targets * (k + 1)

    aic = 2 * total_params - 2 * log_likelihood
    bic = total_params * np.log(n) - 2 * log_likelihood

    return aic, bic


def evaluate2(model, X_train, X_test, y_train, y_test, model_name="Model"):

    y_train_pred = model.predict(X_train)
    y_test_pred  = model.predict(X_test)
    n_features = X_train.shape[1]


    # --- TRAIN METRICS ---
    train_r2   = r2_score(y_train, y_train_pred)
    train_mse  = MSE(y_train, y_train_pred)
    train_adj  = adjusted_r2(y_train, y_train_pred, n_features)
    train_aic, train_bic = calculate_aic_bic_multi(y_train, y_train_pred, n_features)

    # --- TEST METRICS ---
    test_r2    = r2_score(y_test, y_test_pred)
    test_mse   = MSE(y_test, y_test_pred)
    test_adj   = adjusted_r2(y_test, y_test_pred, n_features)
    test_aic, test_bic = calculate_aic_bic_multi(y_test, y_test_pred, n_features)

    # AIC/BIC на тесте обычно не считают, но для симметрии можно оставить или убрать

    # Вывод
    print(f"Метрики {model_name} (признаков: {n_features}):\n")
    print('Train')
    print(f"  R2       = {train_r2:.4f}")
    print(f"  Adj R2   = {train_adj:.4f}")
    print(f"  MSE      = {train_mse:.3e}") # Используем e-format, т.к. числа большие
    print(f"  RMSE     = {train_mse**0.5:.3f}")
    print(f"  AIC      = {train_aic:.2f}")
    print(f"  BIC      = {train_bic:.2f}")

    print("-" * 25)
    print('Test')
    print(f"  R2       = {test_r2:.4f}")
    print(f"  Adj R2   = {test_adj:.4f}")
    print(f"  MSE      = {test_mse:.3e}")
    print(f"  RMSE     = {test_mse**0.5:.3f}")
    print(f"  AIC      = {test_aic:.2f}")
    print(f"  BIC      = {test_bic:.2f}")

In [62]:
from itertools import combinations


class L0RegressionBruteForce:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.best_model = None
        self.best_features_indices = None
        self.best_loss = float('inf')

    def fit(self, X, y):
        n_features = X.shape[1]
        indices = range(n_features)

        # Перебор всех возможных комбинаций признаков (от 1 до всех)
        for k in range(1, n_features + 1):
            for subset_indices in combinations(indices, k):
                subset_indices = list(subset_indices)

                # Берем только выбранные признаки
                X_subset = X[:, subset_indices]
                model = LinearRegression()
                model.fit(X_subset, y)

                # Предсказываем на трейне для расчета MSE
                y_pred = model.predict(X_subset)
                mse = MSE(y, y_pred)

                # Считаем Loss L0: MSE + alpha * k
                # k = len(subset_indices) - количество ненулевых весов
                current_loss = mse + self.alpha * k

                # Если нашли лучший вариант - сохраняем
                if current_loss < self.best_loss:
                    self.best_loss = current_loss
                    self.best_model = model
                    self.best_features_indices = subset_indices

        return self

    def predict(self, X):
        # Применяем модель только к лучшим признакам
        X_subset = X[:, self.best_features_indices]
        return self.best_model.predict(X_subset)


# --- ЭКСПЕРИМЕНТЫ ---

# Поскольку MSE в этой задаче огромный (порядка 10^11),
# штраф alpha тоже должен быть большим, чтобы влиять на выбор (иначе он потеряется на фоне MSE).

alphas = [0, 1e8, 2e8, 3e8, 5e8, 1e9, 5e9, 5e10]

print("======= ЭКСПЕРИМЕНТ С L0 РЕГУЛЯРИЗАЦИЕЙ =======\n")

for alpha in alphas:
    l0_model = L0RegressionBruteForce(alpha=alpha)
    l0_model.fit(X_train_scaled, y_train)

    # Какие признаки выбрались?
    selected_indices = l0_model.best_features_indices
    selected_features = X_train_num.columns[selected_indices].tolist()

    print(f"Alpha (Lambda): {alpha:.0e}")
    print(f"Выбрано признаков: {len(selected_features)}")
    print(f"Признаки: {selected_features}")

    evaluate2(l0_model, X_train_scaled, X_test_scaled, y_train, y_test, model_name=f"L0 (alpha={alpha:.0e})")
    print("\n" + "="*50 + "\n")

======= ЭКСПЕРИМЕНТ С L0 РЕГУЛЯРИЗАЦИЕЙ =======

Alpha (Lambda): 0e+00
Выбрано признаков: 8
Признаки: ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'torque', 'seats', 'max_torque_rpm']
Метрики L0 (alpha=0e+00) (признаков: 8):

Train
  R2       = 0.6016
  Adj R2   = 0.6010
  MSE      = 1.142e+11
  RMSE     = 337936.895
  AIC      = 165284.78
  BIC      = 165344.83
-------------------------
Test
  R2       = 0.5986
  Adj R2   = 0.5953
  MSE      = 2.307e+11
  RMSE     = 480356.712
  AIC      = 29020.45
  BIC      = 29064.62


Alpha (Lambda): 1e+08
Выбрано признаков: 7
Признаки: ['year', 'km_driven', 'engine', 'max_power', 'torque', 'seats', 'max_torque_rpm']
Метрики L0 (alpha=1e+08) (признаков: 8):

Train
  R2       = 0.6015
  Adj R2   = 0.6009
  MSE      = 1.142e+11
  RMSE     = 337979.826
  AIC      = 165286.26
  BIC      = 165346.31
-------------------------
Test
  R2       = 0.5985
  Adj R2   = 0.5953
  MSE      = 2.308e+11
  RMSE     = 480401.295
  AIC      = 29020.63
  BI

Сначала бросается в глаза, то что необходим очень большой alpha. Далее видно, что уменьшение кол-ва признаков также снижает все наши метрики - включая R2 adjusted(который добавляет штраф за кол-во признаков)


AIC и BIC - также ухудшались(увеличивальсь, чем метрика больше тем хуже) с уменьшением кол-ва признаков

# **Часть 3 (0.5 балла) | Добавляем категориальные фичи**

Попробуем для улучшения модели дать ей больше признаков. Добавим категориальные фичи.

За эту часть можно набрать 0.5 основных балла.



### **Задание 19 (0.1 балла)**

Проанализируйте столбец `name`. Очевидно, что эта переменная является категориальной, однако категорий в ней много.

- [ ] Предобработайте столбец `name`, чтобы избежать его удаления

In [63]:
df_train.loc[:, ['name']] = df_train.name.apply(lambda x : x.lower().split()[0])
df_test.loc[:, ['name']] = df_test.name.apply(lambda x : x.lower().split()[0])

In [64]:
df_train.name.unique(), len(df_train.name.unique())

(array(['maruti', 'skoda', 'hyundai', 'toyota', 'ford', 'renault',
        'mahindra', 'honda', 'chevrolet', 'fiat', 'datsun', 'tata', 'jeep',
        'mercedes-benz', 'mitsubishi', 'audi', 'volkswagen', 'bmw',
        'nissan', 'lexus', 'jaguar', 'land', 'mg', 'volvo', 'daewoo',
        'kia', 'force', 'ambassador', 'isuzu', 'peugeot'], dtype=object),
 30)

In [65]:
df_test.name.unique(), len(df_test.name.unique())

(array(['mahindra', 'tata', 'honda', 'chevrolet', 'maruti', 'hyundai',
        'lexus', 'ford', 'jeep', 'renault', 'toyota', 'skoda', 'bmw',
        'datsun', 'volvo', 'nissan', 'volkswagen', 'ambassador', 'jaguar',
        'mercedes-benz', 'mitsubishi', 'audi', 'opel', 'fiat', 'ashok'],
       dtype=object),
 25)

Всего 30 брендов в трейне. В тесте 25 брендов

### **Задание 20 (0.1 балла)**

- [ ] Закодируйте категориалльные фичи и ``seats`` методом OneHot-кодирования.

In [66]:
cat_cols = list(df_train.select_dtypes(object).columns) + ['seats']
num_cols = list(df_train.select_dtypes([int, float]).columns.drop(['seats', 'selling_price']))

In [67]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'),
         cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)

X_train_preprocessed = preprocessor.fit_transform(df_train.drop('selling_price', axis=1))
X_test_preprocessed  = preprocessor.transform(df_test.drop('selling_price', axis=1))

cat_cols = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols)
all_cols = list(cat_cols) + num_cols

X_train_cat = pd.DataFrame(X_train_preprocessed, columns=all_cols, index=df_train.index)
X_test_cat  = pd.DataFrame(X_test_preprocessed,  columns=all_cols,  index=df_test.index)

print(f"Размер после предобработки: {X_train_cat.shape}")
print("Признаки включены:")
print(f"  • Категориальные (OneHot): {len(cat_cols)}")
print(f"  • Числовые (StandardScaler): {len(num_cols)}")

Размер после предобработки: (5840, 54)
Признаки включены:
  • Категориальные (OneHot): 47
  • Числовые (StandardScaler): 7


/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [68]:
X_train_cat.head(3)

,name_audi,name_bmw,name_chevrolet,name_daewoo,name_datsun,name_fiat,name_force,name_ford,name_honda,name_hyundai,...,seats_9,seats_10,seats_14,year,km_driven,mileage,engine,max_power,torque,max_torque_rpm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1,1.2,1.0,-0.4,-0.4,0.3,-0.6
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1,0.8,0.4,0.1,0.5,1.0,-1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-0.8,0.9,0.9,-0.1,0.1,0.6,-0.8


In [69]:
model5 = LinearRegression()
model5.fit(X_train_cat, y_train)
evaluate2(model5, X_train_cat, X_test_cat, y_train, y_test, "With categorial features")

Метрики With categorial features (признаков: 54):

Train
  R2       = 0.7762
  Adj R2   = 0.7741
  MSE      = 6.415e+10
  RMSE     = 253277.274
  AIC      = 162008.57
  BIC      = 162375.55
-------------------------
Test
  R2       = 0.7889
  Adj R2   = 0.7768
  MSE      = 1.213e+11
  RMSE     = 348350.560
  AIC      = 28469.81
  BIC      = 28739.73


### **Задание 21 (0.2 балла)**

OHE — базовый алгоритм преобразования категориальных признаков, но и с ним нужно быть аккуратными.

**Ответьте на вопросы:**


* Как корректно работать с OHE преобразованием?
* Почему мы удаляем один столбец?
* Пусть из $n$ признаков мы получили $n-1$ столбец, из которых $k < n -1$ оказались не важными по весам модели. Корректно ли их удалить?


>1. Как корректно работать с OHE?
1.  **Обучать только на Train:** Делаем `.fit()` на тренировочных данных, а к тестовым применяем `.transform()`. Это предотвращает утечку данных.
2.  **Обработка новых значений:** Использовать параметр `handle_unknown='ignore'`, чтобы модель не ломалась, если в тестовой выборке встретится категория, которой не было в обучении (ей просто присвоятся нули). Как ссобственно у нас было с брендами машин

>2. Почему мы удаляем один столбец?

Чтобы избежать **мультиколлинеарности** (ловушка фиктивных переменных).
Если оставить все столбцы, их сумма будет всегда равна 1. Это создает прямую линейную зависимость со свободным членом (Bias) модели, из-за чего невозможно найти единственное математическое решение уравнения регрессии.

>3. Корректно ли удалять неважные столбцы из OHE?

**Да, корректно.**
Удаление столбца означает, что его коэффициент становится равен 0.
Это упрощает модель и делает её более интерпретируемой


### **Задание 22 (0.1 балла)**
Повторим то, что делали на прошлом шаге для моделей на вещественных признаках, однако теперь с моделью `Ridge`.


**Ваша задача:**
- [ ] Переберите параметр регуляризации `alpha` для гребневой (ridge) регрессии с помощью класса `GridSearchCV` В качестве параметров при объявлении GridSearchCV кроме модели укажите метрику качества $R^2$. Кроссвалидируйтесь по 10-ти фолдам.
- [ ] Ответье на вопрос: Удалось ли улучшить качество прогнозов?

In [70]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [71]:
param_grid = {
    'alpha': np.arange(0, 4, 0.1)
}

grid3 = GridSearchCV(
    Ridge(random_state=42, max_iter=5000),
    param_grid,
    cv=10,
    scoring='r2',
    n_jobs=-1
)

grid3.fit(X_train_cat, y_train);

In [72]:
grid3.best_estimator_

,alpha,2.1
,fit_intercept,True
,copy_X,True
,max_iter,5000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [73]:
len(grid3.param_grid['alpha'])

40

In [74]:
best_coefs(grid3.best_estimator_, X_train_cat.columns, n = 20)


------Коэффициенты------
                 Признак  Коэффициент
37  owner_Test Drive Car    2032139.7
28            name_volvo    1456921.8
15            name_lexus     999619.6
1               name_bmw     956338.7
14             name_land     666024.2
18    name_mercedes-benz     572424.5
11           name_jaguar     530320.2
25             name_tata    -494509.5
4            name_datsun    -473326.4
2         name_chevrolet    -467340.5
27       name_volkswagen    -461689.4
5              name_fiat    -438422.5
16         name_mahindra    -420202.3
0              name_audi     418652.3
24            name_skoda    -413942.5
23          name_renault    -409057.8
21           name_nissan    -396419.0
7              name_ford    -392604.9
8             name_honda    -389607.8
9           name_hyundai    -374101.5


In [75]:
evaluate2(grid3.best_estimator_, X_train_cat, X_test_cat, y_train, y_test, "With categorial features and Ridge")

Метрики With categorial features and Ridge (признаков: 54):

Train
  R2       = 0.7687
  Adj R2   = -10245.6880
  MSE      = 6.631e+10
  RMSE     = 257503.251
  AIC      = 162201.84
  BIC      = 162568.83
-------------------------
Test
  R2       = 0.7768
  Adj R2   = -1648.9875
  MSE      = 1.283e+11
  RMSE     = 358211.644
  AIC      = 28525.64
  BIC      = 28795.56


По сравнению с моделями только с числовыми признаками - да мы сильно улучшили качество. По сравнению с моделью с категаариальными признаками но без Ridge - нет, стало немного хуже

# **Часть 4 - бонусная (1 балл) | Feature Engineering**

В этой части домашнего задания вам предлагается проявить свою креативность для улучшения прогноза модели. Любые другие модели, кроме различных форм линейной (или полиномиальной) регресси, использовать запрещается. А значит, придется работать с признаками

**Что можно попробовать сделать?** (каждый пункт по 0.4 балла, но не больше 1-х балла в сумме)

1.   *Сгенерировать новые признаки на основе уже существующих:*
    * посчитать произведения // частные признаков (кажется, что посчитать число "лошадей" на литр объема может быть полезно);
    * имеет смысл обратить внимание на визуализации в части с EDA (к примеру, зависимость цены от года выглядит квадратичной, а не линейной; значит, квадрат года нам, скорее всего, принесет больше пользы)

2.   *Добыть новые признаки:*
    * имеем название автомобиля, которое никак не используем (можно спарсить инфу о классе автомобиля или каких-то специфических опциях)
    * можно добавить пороговые признаки вроде "владелец третий или больше" и объединить признаки в некоторые осмысленные правила, например "первый или второй владелец и продавец официальный дилер" (подбирать пороги удобно по диаграммам рассеяния)

3.   *Поработать с уже имеющимися:*
    * далеко не факт, что заполнить пропуск медианой было лучшей идеей (как минимум, можно добавить dummy-столбец для модели, сигнализирующий, что раньше на месте медианы был пропуск -- там где он был, конечно); попробуйте другие способы филлинга;
    * мы не анализировали, есть ли в данных выбросы => никак выбросы не обрабатывали; наиболее простым и, тем не менее, довольно полезным вариантом нахождения выбросов могут послужить boxplot'ы для каждого столбца; что делать с выбросами думайте сами :) -- вариантов довольно много
    * мы толком не смотрели на таргет сам по себе; в нем тоже могут быть неожиданности -- стоит хотя бы проверить
    * можно заметить, что некоторые признаки распределены совсем не нормально; возможно их стоит отлогарифмировать

И так далее...

Feel Free to Try!

На eda я видел, что распределение скошенное у пробега и цены. Возьмём логарифм, чтобы было похоже на нормальное распределение.

In [76]:
y_train_log = np.log1p(df_train['selling_price'])
y_test_log  = np.log1p(df_test['selling_price'])

df_train['km_driven_log'] = np.log1p(df_train['km_driven'])
df_test['km_driven_log']  = np.log1p(df_test['km_driven'])

cat_cols = list(df_train.select_dtypes(object).columns) + ['seats']
num_cols = list(df_train.select_dtypes([int, float]).columns.drop(['seats', 'selling_price']))


preprocessor_log = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)

X_train_final = preprocessor_log.fit_transform(df_train.drop('selling_price', axis=1))
X_test_final  = preprocessor_log.transform(df_test.drop('selling_price', axis=1))

cat_features_final = preprocessor_log.named_transformers_['cat'].get_feature_names_out(cat_cols)
all_columns_final = list(cat_features_final) + num_cols

X_train_final_df = pd.DataFrame(X_train_final, columns=all_columns_final, index=df_train.index)
X_test_final_df  = pd.DataFrame(X_test_final,  columns=all_columns_final,  index=df_test.index)

/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [77]:
model_final = LinearRegression()
model_final.fit(X_train_final_df, y_train_log)

y_train_pred_log = model_final.predict(X_train_final_df)
y_test_pred_log  = model_final.predict(X_test_final_df)

y_train_pred = np.expm1(y_train_pred_log)
y_test_pred  = np.expm1(y_test_pred_log)

print("ФИНАЛЬНАЯ МОДЕЛЬ С ЛОГАРИФМОМ ЦЕНЫ")
evaluate2(model_final, X_train_final_df, X_test_final_df, y_train_log, y_test_log, "Log target + km_log")

ФИНАЛЬНАЯ МОДЕЛЬ С ЛОГАРИФМОМ ЦЕНЫ
Метрики Log target + km_log (признаков: 55):

Train
  R2       = 0.8845
  Adj R2   = 0.8834
  MSE      = 6.750e-02
  RMSE     = 0.260
  AIC      = 942.65
  BIC      = 1316.31
-------------------------
Test
  R2       = 0.9134
  Adj R2   = 0.9084
  MSE      = 6.194e-02
  RMSE     = 0.249
  AIC      = 168.34
  BIC      = 443.18


Вернём предсказыный таргет в первоначальную величину:

In [78]:
print(f"\nR2 на тесте : {r2_score(y_test, y_test_pred):.4f}")
print(f"RMSE на тесте: {MSE(y_test, y_test_pred)**0.5:.0f}")


R2 на тесте : 0.9166
RMSE на тесте: 218909


Итоговое качество в R2 у нас получилось 0.9166!
Это хороший результат

# **Часть 4. | Бизнесовая (0.5 балла)**

### **Задание 23 (0.25 балла)**

В мире бизнеса очень важно давать оценку качества модели понятную бизнесу, поэтому иногда заказчики приходят с кастомными метриками. Попробуем сделать такую для нашей задачи.

**Описание метрики:**

Среди всех предсказанных цен на авто нужно посчитать долю прогнозов, отличающихся от реальных цен на эти авто не более чем на 10% (в одну или другую сторону)

**Ваша задача:**

- [ ] Реализуйте метрику `business_metric`
- [ ] Посчитайте метрику для всех обученных моделей и определеите, какаю лучше всего решает задачу бизнеса

In [79]:
def business_metric(y_true, y_pred, tolerance=0.10):

    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    relative_error = np.abs(y_pred - y_true) / y_true
    within_tolerance = relative_error <= tolerance

    return np.mean(within_tolerance)


In [80]:
for model_v in [model1, model2, model3, model4, grid1.best_estimator_, grid2.best_estimator_]:
    y_pred = model_v.predict(X_test_num)
    print(business_metric(y_test, y_pred))

0.24
0.0
0.0
0.0
0.0
0.0


/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(

In [81]:
for model_v in [model5, grid3.best_estimator_]:
    y_pred = model_v.predict(X_test_cat)
    print(business_metric(y_test, y_pred))

0.326
0.301


In [82]:
y_test_pred_log  = model_final.predict(X_test_final_df)
y_test_pred  = np.expm1(y_test_pred_log)
print(business_metric(y_test, y_test_pred))

0.342


Лучше всех - моя последняя модель. Но как будто качество всё равно так себе.

### **Задание 24 (0.25 балла)**

Но у бизнеса не всегда есть идеи и иногда задача на выбор метрики делегируется вам.

**Задание:**

- [ ] Придумайте и реализуйте другую кастомную метрику с учетом того, что модель не должна сильно ошибаться в прогнозе, но недопрогноз для модели, согласно мнению бизнеса, хуже, чем перепрогноз.
- [ ] Посчитайте метрику для всех обученных моделей и определеите, какаю лучше всего решает задачу бизнеса



In [83]:
def asymmetric_metric(y_true, y_pred, weight=2.0):
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    relative_error = np.abs(y_pred - y_true) / y_true
    under = y_pred - y_true < 0                     # недопрогноз
    loss = np.where(under,
                    relative_error * weight,   # штрафуем в weight раз сильнее
                    relative_error)           # обычный штраф за перепроғноз

    return loss.mean()

In [84]:
for model_v in [model1, model2, model3, model4, grid1.best_estimator_, grid2.best_estimator_]:
    y_pred = model_v.predict(X_test_num)
    print(asymmetric_metric(y_test, y_pred))

0.9462011242060937
24941.7249994957
24941.234714211012
22310.255442714624
12642.566697002168
12642.566697002168


/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/Users/vovazinev/Desktop/ML/01_HW/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(

In [85]:
for model_v in [model5, grid3.best_estimator_]:
    y_pred = model_v.predict(X_test_cat)
    print(asymmetric_metric(y_test, y_pred))

0.6339173882263927
0.648487838548386


In [86]:
print(asymmetric_metric(y_test, y_test_pred))

0.28258289915123713


Тут чем меньше asymmetric_metric тем лучше. Опять таки победила финальная модель

# **<font color="green">Часть 5 | Создание интерактивного приложения на Streamlit (3 балла)</font>**

Вам необходимо создать интерактивное приложение на Streamlit, которое будет:

- Показывать основные информативные графики/гистограммы в рамках EDA (1 балл)
- На вход запрашивать csv-файл с признаками объектов или запрашивать признаки объекта в окошках для ввода, и применять на поступивших объектах модель (1 балл)
- Визуализировать веса обученной модели (1 балл)

In [ ]:
import pickle
import os
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression


df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')

def prepare_data(df):
    def clean_num(x):
        if pd.isna(x): return np.nan
        x = str(x).lower().replace('kmpl', '').replace('km/kg', '').replace('cc', '').replace('bhp', '').strip()
        return float(x) if x else np.nan

    def parse_torque(t):
        if pd.isna(t): return pd.Series([np.nan, np.nan])
        t = str(t).lower().replace(',', '').replace(' ', '')
        import re
        trq = re.search(r"([\d\.]+)", t)
        rpm = re.search(r"(\d+)", t) # Просто берем первые цифры
        return pd.Series([float(trq.group(1)) if trq else np.nan, float(rpm.group(1)) if rpm else np.nan])

    df = df.copy()
    df['mileage'] = df['mileage'].apply(clean_num)
    df['engine'] = df['engine'].apply(clean_num)
    df['max_power'] = df['max_power'].apply(clean_num)


    cols = ['mileage', 'engine', 'max_power', 'seats']
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(df[c].median())

    df[['engine', 'seats']] = df[['engine', 'seats']].astype(int)

    df["brand"] = df["name"].str.split(" ", n=1).str[0]
    return df

df_clean = prepare_data(df_train)
df_clean['km_driven_log'] = np.log1p(df_clean['km_driven']) # Логарифм

cat_cols = ['fuel', 'seller_type', 'transmission', 'owner', 'seats', 'brand']
num_cols = ['year', 'km_driven_log', 'mileage', 'engine', 'max_power']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_cols),
        ('num', StandardScaler(), num_cols)
    ],
    verbose_feature_names_out=False
)


X = preprocessor.fit_transform(df_clean)
y = np.log1p(df_clean['selling_price'])

model = LinearRegression()
model.fit(X, y)

print(f"✅ Модель обучена на {X.shape[1]} признаках.")

# Сохраняем
packet = {
    'model': model,
    'preprocessor': preprocessor,
    'medians': df_clean[num_cols + ['seats']].median().to_dict()
}

os.makedirs('models', exist_ok=True)
with open('models/final_pipeline.pkl', 'wb') as f:
    pickle.dump(packet, f)

print("✅ Сохранено в models/final_pipeline.pkl")

✅ Модель обучена на 52 признаках.
✅ Сохранено в models/final_pipeline.pkl


In [95]:
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

# Сохраняем в файл
df_test.to_csv('cars_test.csv', index=False)

# **Часть 6 | Оформление репозитория и оценка сервиса (3 балла)**

**Результаты вашей работы** необходимо разместить в своём GitHub-репозитории. В этот же репозиторий позже будет добавлена вторая часть.

Под результатами первой части понимаем следующее:

---

### Обязательные файлы:

1. **`.ipynb`-ноутбук** со всеми экспериментами:

   * полный EDA,
   * все шаги препроцессинга,
   * обучение и сравнение моделей,
   * сохранённые output’ы.

2. **`.pickle`-файл**, содержащий:

   * обученную модель (или пайплайн `scaler + model`);
   * параметры скейлинга;
   * любые числовые объекты, необходимые для инференса внутри Streamlit-приложения.

3. **`.md`-файл с выводами** о проделанной работе:

   * что было сделано (краткое описание каждого этапа);
   * какие результаты были получены (метрики + интерпретация);
   * что дало наибольший прирост качества;
   * что сделать не удалось и почему (это нормально и даже полезно);
   * **оценка разработанного сервиса**:

     * насколько приложение удобно в использовании;
     * что получилось визуализировать хорошо, а что — менее удачно;
     * какие ограничения или проблемы вы заметили;
     * какие улучшения планируете в следующей итерации.

По результатам второй части, ждем:

4. **Streamlit-приложение** (например, `app.py`), которое можно запустить командой:

   ```bash
   streamlit run app.py
   ```

В приложении должно быть:

* отображение ключевых графиков EDA;
* ввод данных (CSV или ручной ввод);
* применение модели;
* визуализация весов/коэффициентов.

**И ссылку на приложение в StreamlitHub.**


### **За что могут быть сняты баллы:**

* отсутствие `.pickle` с моделью / пайплайном;
* слабая или неполная аналитика в `.md`-файле;
* беспорядок в ноутбуке или неясная логика вычислений;
* отсутствие возможности запустить Streamlit-приложение.
